### Geometric Computer Vision, Skoltech, 2021
#### Alexey Artemov, Aleksandr Safin


In [1]:
import torch
import numpy as np
import k3d
import imageio
from skimage import measure

### TSDF Fusion in brief

For every new observation $O_{i+1}$
- RGBD to point cloud
- TSDF creation from one RGBD
- Integrate TSDF for $O_{i+1}$ with current TSDF
   - You need to weight appropriately 

Afterall run marshing cubes on your TSDF

### Important Notes:

Volume bounds are in meters, depth also should be specified in meters

You should use the following intrinsics matrix for the camera:
```
array([[525. ,   0. , 319.5],
       [  0. , 525. , 239.5],
       [  0. ,   0. ,   1. ]])
```

ans with width = 640 and height = 480.

### Redwood Data

Here we will use a part of [Redwood indoor RGBD dataset](http://redwood-data.org/indoor_lidar_rgbd/)

RGBD are in ``sample_data/redwood_samples`.

Extrinsics are in `sample_data/poses.npy`, use `np.load` to open it.

In [3]:
class TSDFVolume:
    """
        Volumetric TSDF Fusion of RGB-D Images.
    """
    def __init__(self, vol_bnds, voxel_size):
        """Constructor.
        Args:
          vol_bnds (ndarray): An ndarray of shape (3, 2). Specifies the
            xyz bounds (min/max) in meters.
          voxel_size (float): The volume discretization in meters.
        """
        vol_bnds = np.asarray(vol_bnds)
        assert vol_bnds.shape == (3, 2), "[!] `vol_bnds` should be of shape (3, 2)."

        # Define voxel volume parameters
        self._vol_bnds = vol_bnds
        self._voxel_size = float(voxel_size)
        self._trunc_margin = 10 * self._voxel_size  # truncation on SDF

        # Adjust volume bounds and ensure C-order contiguous
        self._vol_dim = np.ceil((self._vol_bnds[:,1]-self._vol_bnds[:,0])/self._voxel_size).copy(order='C').astype(int)
        self._vol_bnds[:,1] = self._vol_bnds[:,0] + self._vol_dim*self._voxel_size
        self._vol_origin = self._vol_bnds[:,0].copy(order='C').astype(np.float32)

        print("Voxel volume size: {} x {} x {} - # points: {:,}".format(
          self._vol_dim[0], self._vol_dim[1], self._vol_dim[2],
          self._vol_dim[0]*self._vol_dim[1]*self._vol_dim[2])
        )

        # Initialize voxel volume
        self._tsdf_vol = np.ones(self._vol_dim).astype(np.float32)
        # for computing the cumulative moving average of observations per voxel
        self._weight_vol = np.zeros(self._vol_dim).astype(np.float32)

        # Get voxel grid coordinates
        xv, yv, zv = np.meshgrid(
            range(self._vol_dim[0]),
            range(self._vol_dim[1]),
            range(self._vol_dim[2]),
            indexing='ij'
        )
        self.vox_coords = np.concatenate([
            xv.reshape(1,-1),
            yv.reshape(1,-1),
            zv.reshape(1,-1)
        ], axis=0).astype(int).T

    @staticmethod
    def vox2world(vol_origin, vox_coords, vox_size):
        """
            Convert voxel grid coordinates to world coordinates.
        """
        vol_origin = vol_origin.astype(np.float32)
        vox_coords = vox_coords.astype(np.float32)
        cam_pts = np.empty_like(vox_coords, dtype=np.float32)
        for i in range(vox_coords.shape[0]):
            for j in range(3):
                cam_pts[i, j] = vol_origin[j] + (vox_size * vox_coords[i, j])
        return cam_pts

    @staticmethod
    def cam2pix(cam_pts, intr):
        """
            Convert camera coordinates to pixel coordinates.
        """
        intr = intr.astype(np.float32)
        fx, fy = intr[0, 0], intr[1, 1]
        cx, cy = intr[0, 2], intr[1, 2]
        pix = np.empty((cam_pts.shape[0], 2), dtype=np.int64)
        for i in range(cam_pts.shape[0]):
            pix[i, 0] = int(np.round((cam_pts[i, 0] * fx / cam_pts[i, 2]) + cx))
            pix[i, 1] = int(np.round((cam_pts[i, 1] * fy / cam_pts[i, 2]) + cy))
        return pix

    @staticmethod
    def integrate_next_obs_tsdf(tsdf_vol, dist, w_old, obs_weight):
        """
            Integrate the TSDF volume.
            Agrs:
                tsdf_vol: Accumulated TSDF
                dist: TSDF for current observation
            Returns:
                tsdf_vol_int: Fused new accumulated TSDF
                w_new: updated new W_{i+1}
        """
        tsdf_vol_int = np.empty_like(tsdf_vol, dtype=np.float32)
        w_new = np.empty_like(w_old, dtype=np.float32)

        """
            Update tsdf_vol_int and w_new
        """
        return tsdf_vol_int, w_new

    def integrate(self, color_im, depth_im, cam_intr, cam_pose, obs_weight=1.):
        """Integrate an RGB-D frame into the TSDF volume.
        Args:
          color_im (ndarray): An RGB image of shape (H, W, 3).
          depth_im (ndarray): A depth image of shape (H, W).
          cam_intr (ndarray): The camera intrinsics matrix of shape (3, 3).
          cam_pose (ndarray): The camera pose (i.e. extrinsics) of shape (4, 4).
          obs_weight (float): The weight to assign for the current observation.
        """
        im_h, im_w = depth_im.shape

        # Convert voxel grid coordinates to pixel coordinates
        cam_pts = self.vox2world(self._vol_origin, self.vox_coords, self._voxel_size)
        cam_pts = rigid_transform(cam_pts, np.linalg.inv(cam_pose))
        pix_z = cam_pts[:, 2]
        pix = self.cam2pix(cam_pts, cam_intr)
        pix_x, pix_y = pix[:, 0], pix[:, 1]

        # Eliminate pixels outside view frustum
        valid_pix = np.logical_and(pix_x >= 0,
                  np.logical_and(pix_x < im_w,
                  np.logical_and(pix_y >= 0,
                  np.logical_and(pix_y < im_h,
                  pix_z > 0))))
        depth_val = np.zeros(pix_x.shape)
        depth_val[valid_pix] = depth_im[pix_y[valid_pix], pix_x[valid_pix]]

        # Integrate TSDF
        depth_diff = depth_val - pix_z
        valid_pts = np.logical_and(depth_val > 0, depth_diff >= -self._trunc_margin)
        dist = np.minimum(1, depth_diff / self._trunc_margin)
        valid_vox_x = self.vox_coords[valid_pts, 0]
        valid_vox_y = self.vox_coords[valid_pts, 1]
        valid_vox_z = self.vox_coords[valid_pts, 2]
        w_old = self._weight_vol[valid_vox_x, valid_vox_y, valid_vox_z]
        tsdf_vals = self._tsdf_vol[valid_vox_x, valid_vox_y, valid_vox_z]
        valid_dist = dist[valid_pts]
        
        # Here you weight appropriately accumulated TSDF and TSDF for current observation
        tsdf_vol_new, w_new = self.integrate_next_obs_tsdf(tsdf_vals, valid_dist, w_old, obs_weight)
        
        self._weight_vol[valid_vox_x, valid_vox_y, valid_vox_z] = w_new
        self._tsdf_vol[valid_vox_x, valid_vox_y, valid_vox_z] = tsdf_vol_new

    def get_volume(self):
        return self._tsdf_vol

    def get_mesh(self):
        """
            Compute a mesh from the voxel volume using marching cubes.
            Returns:
                verts
                faces
                norms
        """
        tsdf_vol = self.get_volume()

        # Marching cubes
        verts, faces, norms, vals = measure.marching_cubes_lewiner(tsdf_vol, level=0)
        verts_ind = np.round(verts).astype(int)
        
        """
            Here in verts you have coordinates of voxel grid, so you need to tranfrom them to world coordinates
            Hint: think about the offset and voxel size
        """
#         verts =

        return verts, faces, norms


def rigid_transform(xyz, transform):
    """
        Applies a rigid transform to an (N, 3) pointcloud.
    """
    xyz_h = np.hstack([xyz, np.ones((len(xyz), 1), dtype=np.float32)])
    xyz_t_h = np.dot(transform, xyz_h.T).T
    return xyz_t_h[:, :3]


def get_view_frustum(depth_im, cam_intr, cam_pose):
    """
        Get corners of 3D camera view frustum of depth image
        
        Args:
            depth_im: Depth
            cam_intr: Camera intrinsics. 3x3 matrix
            cam_pose: Camera poses. 4x4 matrix
            
        Return:
            view_frustrum_points: 5x3 matrix. Contains world coordinates positions of the 5 points \
                                  of view frustrum associated with the camera.
    """

    ### Create array of that 5 points in camera coordinates, in pixels. You will have 5x2 matrix
    
    ### Then transform them into 3d space, in camera coordinates system
    
    ### Then convert to world coordinates
    view_frustrum_points = np.seros(5,3) # just a dummy
    
    return view_frustrum_points


To obtain TSDF $D_{i+1}$ for all observations from $1$ up to $i+1$, the TSDF from $D_{i}$ from the previous step $i$, is fused with TSDF for observation $O_{i+1}$ as follows.

$$
D_{i+1} = \frac{W_i D_i + w_{i+1} d_{i+1}}{W_{i+1}},
$$
where $w_{i+1}$ is weight for TSDF constructed from the observation $O_{i+1}$.

The weight $W_{i+1}$ of accumulated TSDF for is updated as:
$$
W_{i+1} = W_{i} + w_{i+1}
$$

### Warmup look at the data

### Project depth to world as point cloud and visualize it